# The map of Taxable Income per Riding for 2015 elections

Legend:

* Red - Liberal Party of Canada

* Blue - Conservative Party of Canada

* Orange - New Democratic Party

* Light Blue - Bloc Québécois

* Green  - Green Party of Canada

The relative size of the circle corresponds to the absolute amount of Taxable Income for a Riding.

In [1]:
# Dependencies and initialization
import gmaps
import pandas as pd

# Google developer API key
from api_keys import g_key

# Data sources
geodata_csv = 'geodata.csv' # FED Coordinates (key: 'FED Id')
parties_csv = '../data/winning_parties_2015.csv' # List of winning parties (key: 'Custom Name')
results_csv = '../data/final_elected.csv' # List of election results by FED (keys: 'FED ID', 'Party Won')
income_csv = '../data/income.csv' # FED Income Data (key: 'FED ID')

# Conclusions

Here are some conclusions, that one could make after a glance on the map:

1. "Red" centers of large agglomerations: Toronto, Montreal, Ottawa, Vancouver, Calgary. Might correlate with very high property prices and property taxes (https://www.livingin-canada.com/house-prices-canada.html). Also there might be a correlation with a demographic shift towards elder population, who purchased property in these areas many years ago in different economy.

Ontario | Montreal
------- | --------
![Ontario](Income_Ontario.png)|![Montreal](Income_Montreal.png)

2. In support of high property prices theory let's check out Vancouver area. Their support for NDP is third (percentwise) after NWT and Nunavut. Combined with the Liberals they overcome Conservatives 2-by-1. This might indicate general discontent of social inequality by downtown residents.

Vancouver | Alberta
--------- | --------
![Vancouver](Income_Vancouver.png)|![Alberta](Income_Alberta.png)

3. The opposite example that proves the point is the province of Alberta with it's natural resources and the highest GDP per capita in Canada (https://en.wikipedia.org/wiki/Alberta#Economy). 59.6% votes in favour of the Conservatives vs. 36.1% for Liberals and NDP, practically 2-by-1, diametrally opposed picture compared to BC. Though, central ridings of Edmonton and Calgary are Red and Orange.

4. Separatist sentiments are clearly seen near Montreal, the only ridings where Bloc Québécois won elections.

5. NDP support in Windsor area (ON) might indicate depressive economy.


In [2]:
# Read data
geodata_df = pd.read_csv(geodata_csv).loc[:, ['FED Id', 'FED Name', 'Latitude', 'Longitude']]
parties_df = pd.read_csv(parties_csv).loc[:, ['Id', 'Custom Name']]
results_df = pd.read_csv(results_csv).loc[:, ['FED ID', 'Party Won']]
income_df = pd.read_csv(income_csv, encoding='windows-1252').loc[:, ['Prov/Terr', 'FED ID', 'Total', 'Taxable Income']]
income_df = income_df[income_df['Prov/Terr'] != 'TOTAL']
income_df.count()

Prov/Terr         338
FED ID            338
Total             338
Taxable Income    338
dtype: int64

In [3]:
# Some useful functions

# From the pd.Series s with numeric values calculate scale for each point
def make_scale_list(s, bin_count=10):
    interval_index = pd.interval_range(start=s.min() * 0.999, end=s.max(), periods=bin_count)
    return [1 + 1 * int(interval_index.get_loc(val)) for val in s]

# From pd.Series s with Party Ids make a list of party colors
def make_color_list(s):
    # Source: https://www.rapidtables.com/web/color/RGB_Color.html
    c_blue      = (51, 51, 255)
    c_lightblue = (51, 153, 255)
    c_green     = (0, 204, 0)
    c_orange    = (255, 165, 0)
    c_red       = (255, 0, 0)

    # Source: https://en.wikipedia.org/wiki/Political_colour
    party_colors = {
        'CPC': c_blue,
        'BC' : c_lightblue,
        'GRN': c_green,
        'NDP': c_orange,
        'LIB': c_red
    }

    return s.map(lambda id: party_colors[id]).tolist()

In [4]:
# Create data set

df = pd.merge(geodata_df, results_df, how='left', left_on='FED Id', right_on='FED ID')
df = pd.merge(df, parties_df, how='left', left_on='Party Won', right_on='Custom Name')
df = pd.merge(df, income_df, how='left', left_on='FED Id', right_on='FED ID')
df.drop(columns=['FED ID_x', 'FED ID_y', 'Custom Name', 'Prov/Terr'], inplace=True)
df.rename(columns={'Id': 'Party Id'}, inplace=True)
#df = df[df['Party Id'] == 'NDP'] # Filter for testing
df.head()

,FED Id,FED Name,Latitude,Longitude,Party Won,Party Id,Total,Taxable Income
0,11001,Cardigan,46.251413,-62.652435,Liberal,LIB,29970,1.103647e+09
1,11002,Charlottetown,46.267987,-63.143870,Liberal,LIB,29650,1.060050e+09
2,24056,Pierrefonds--Dollard,45.491925,-73.855406,Liberal,LIB,86110,3.461175e+09
3,24057,Pontiac,46.760139,-76.594503,Liberal,LIB,88560,3.672320e+09
4,24058,Portneuf--Jacques-Cartier,47.137004,-71.836506,Conservative,CPC,92380,3.892539e+09


In [5]:
# Access maps with unique API key
gmaps.configure(api_key=g_key)

# Customize the size of the figure
figure_layout = {
    'width': '900px',
    'height': '675px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
center=(43.6551165, -79.3869946) # Downtown Toronto

fig = gmaps.figure(layout=figure_layout, center=center, zoom_level=8)

# FED central points
locations = df[['Latitude', 'Longitude']]
hover_text = df['FED Name'].tolist()
fill_color = make_color_list(df['Party Id'])
stroke_color = fill_color
scale = make_scale_list(df.iloc[:, -1])

data_layer = gmaps.symbol_layer(
    locations, # hover_text="hover_text", - doesn't work :(
    fill_color=fill_color, stroke_color=stroke_color,
    scale=scale
)

# Add the layer to the map
fig.add_layer(data_layer)

# Display
fig

Figure(layout=FigureLayout(border='1px solid black', height='675px', margin='0 auto 0 auto', padding='1px', wi…